In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
batch_size = 4
learning_rate = 0.01
num_classes = 10

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)

# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
#                                           shuffle=True)

# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
#                                          shuffle=False)
train_dataset = torchvision.datasets.MNIST(root = './data' , train = True, transform = transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root = './data' , train = False, transform = transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = False)



# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

dataiter = iter(train_loader)
images, labels = next(dataiter)
print(images.shape)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # Change the number of input channels to 1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 150)  # Adjust the input size based on the output shape of the previous layer
        self.fc2 = nn.Linear(150, 84)
        self.fc3 = nn.Linear(84,10)  # num_classes should be set to 10 for MNIST dataset (digits 0-9)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 *4*4)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {i}: {acc} %')


torch.Size([4, 1, 28, 28])
Epoch [1/10], Step [2000/15000], Loss: 0.1327
Epoch [1/10], Step [4000/15000], Loss: 0.0060
Epoch [1/10], Step [6000/15000], Loss: 0.0087
Epoch [1/10], Step [8000/15000], Loss: 0.0523
Epoch [1/10], Step [10000/15000], Loss: 0.0019
Epoch [1/10], Step [12000/15000], Loss: 1.1194
Epoch [1/10], Step [14000/15000], Loss: 1.2416
Epoch [2/10], Step [2000/15000], Loss: 0.0000
Epoch [2/10], Step [4000/15000], Loss: 0.0006
Epoch [2/10], Step [6000/15000], Loss: 0.0804
Epoch [2/10], Step [8000/15000], Loss: 0.0004
Epoch [2/10], Step [10000/15000], Loss: 0.0007
Epoch [2/10], Step [12000/15000], Loss: 0.1645
Epoch [2/10], Step [14000/15000], Loss: 0.0268
Epoch [3/10], Step [2000/15000], Loss: 0.0002
Epoch [3/10], Step [4000/15000], Loss: 0.0044
Epoch [3/10], Step [6000/15000], Loss: 0.0011
Epoch [3/10], Step [8000/15000], Loss: 0.0002
Epoch [3/10], Step [10000/15000], Loss: 0.0100
Epoch [3/10], Step [12000/15000], Loss: 0.0003
Epoch [3/10], Step [14000/15000], Loss: 0.000

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import PIL.Image as Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  # Change the number of input channels to 1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 200)  # Adjust the input size based on the output shape of the previous layer
        self.fc2 = nn.Linear(200, 84)
        self.fc3 = nn.Linear(84, 10)  # num_classes should be set to 10 for MNIST dataset (digits 0-9)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 4 * 4)  # -> n, 400
        x = F.relu(self.fc1(x))  # -> n, 120
        x = F.relu(self.fc2(x))  # -> n, 84
        x = self.fc3(x)  # -> n, 10
        return x

def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image).unsqueeze(0)
    return image

def predict_digit(image, model):
    image = image.to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

model = ConvNet().to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

# Load and preprocess the image from your phone
image_path = [
    '/content/0.jpg','/content/0_2.png', '/content/1.jpg',
    '/content/2.jpg','/content/2_1.png','/content/3.jpg',
    '/content/3.png','/content/4.jpg','/content/5.jpg',
    '/content/5_2.jpg','/content/5_3.jpg','/content/6.jpg',
    '/content/9_2.png','/content/8.png','/content/4_1.jpg'
]
corect = [0,0,1,2,2,3,3,4,5,5,5,6,9,8,4]
noc = 0
nos = 0
for i in range(len(corect)):
  image = Image.open(image_path[i])
  preprocessed_image = preprocess_image(image)

  # Make the prediction
  predicted_digit = predict_digit(preprocessed_image, model)
  print(f'Predicted digit: {predicted_digit}       Actual digit: {corect[i]}')
  if (corect[i] == predicted_digit):
    noc = noc + 1
    print(noc)
acc = 100.0 * (noc / len(corect))
print(f'Accuracy of the network: {acc} %')